<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/No_supervisado_Recomendador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [ ]:
audio = pd.read_csv('audio_features.csv',encoding='latin-1')
song = pd.read_csv('song_genres.csv',encoding='latin-1')
summary = pd.read_csv('summary.csv')

In [ ]:
def text_clean(text, pattern="[^a-zA-Z0-9 ]"):
    text_clean = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    text_clean= re.sub(pattern, " ", text_clean.decode("utf-8"), flags=re.UNICODE)
    text_clean = u' '.join(text_clean.lower().split())
    return text_clean

# Audio

In [ ]:
audio.head(3)

,analysis_sample_rate,bit_rate,equal_loudness,length,lossless,replay_gain,sample_rate,codec,downmix,md5_encoded,...,acousticness,aggressiveness,electronicity,happiness,festiveness,relaxation,sadness,brightness,tonality,insrumentalness
0,44100,128056,0,60.081612,0,-14.896189,22050,mp3,mix,9b2010d602420c02fd513c4bd79f0e1a,...,2.266771e-09,0.999996,0.075123,0.683977,0.510561,0.103477,0.004121,0.217572,0.294421,0.002060
1,44100,128056,0,60.029366,0,-4.654913,22050,mp3,mix,b8130f7ef3c9b28b43718311a851401f,...,9.705600e-01,0.000016,0.173669,0.044266,0.002073,0.990112,0.976110,0.394722,0.994963,0.996827
2,44100,128056,0,60.081612,0,-8.621744,22050,mp3,mix,58f66795976e8a247cfc7b236df8ecfa,...,9.027991e-01,0.116882,0.156126,0.461002,0.079624,0.635178,0.797037,0.962334,0.921192,0.001401


In [ ]:
audio.shape

(19664, 23)

In [ ]:
audio['track_id']=audio['track_id'].map(lambda x:x.strip())


In [ ]:
audio.isna().mean()

analysis_sample_rate    0.0
bit_rate                0.0
equal_loudness          0.0
length                  0.0
lossless                0.0
replay_gain             0.0
sample_rate             0.0
codec                   0.0
downmix                 0.0
md5_encoded             0.0
track_id                0.0
danceability            0.0
femaleness              0.0
acousticness            0.0
aggressiveness          0.0
electronicity           0.0
happiness               0.0
festiveness             0.0
relaxation              0.0
sadness                 0.0
brightness              0.0
tonality                0.0
insrumentalness         0.0
dtype: float64

In [ ]:
audio.describe()

,analysis_sample_rate,bit_rate,equal_loudness,length,lossless,replay_gain,sample_rate,danceability,femaleness,acousticness,aggressiveness,electronicity,happiness,festiveness,relaxation,sadness,brightness,tonality,insrumentalness
count,19664.0,19664.000000,19664.0,19664.000000,19664.0,19664.000000,19664.000000,1.966400e+04,1.966400e+04,1.966400e+04,1.966400e+04,19664.000000,1.966400e+04,1.966400e+04,19664.000000,19664.000000,1.966400e+04,1.966400e+04,1.966400e+04
mean,44100.0,110472.998169,0.0,46.808226,0.0,-10.939226,28692.908360,4.360305e-01,5.613689e-01,3.199084e-01,3.601415e-01,0.480301,4.670382e-01,3.964575e-01,0.488716,0.381916,6.016381e-01,5.351372e-01,2.632997e-01
std,0.0,28907.453634,0.0,15.401451,0.0,3.767648,10116.872522,3.839627e-01,3.158331e-01,3.613495e-01,3.704963e-01,0.320995,2.845934e-01,3.080999e-01,0.362255,0.313213,3.613348e-01,3.784324e-01,3.512926e-01
min,44100.0,49978.000000,0.0,0.992630,0.0,-21.987061,22050.000000,3.000001e-14,7.190201e-11,3.000001e-14,3.000001e-14,0.000004,3.000001e-14,1.012808e-07,0.000413,0.000907,5.489374e-07,3.000001e-14,1.000000e-07
25%,44100.0,64062.000000,0.0,30.040817,0.0,-13.598913,22050.000000,5.679470e-02,2.876552e-01,1.142855e-02,3.327452e-02,0.183631,2.199898e-01,9.395351e-02,0.115004,0.090135,2.375204e-01,1.266094e-01,6.027732e-03
50%,44100.0,128056.000000,0.0,60.029366,0.0,-11.689271,22050.000000,3.284215e-01,5.931417e-01,1.211191e-01,1.873492e-01,0.440004,4.559144e-01,3.610335e-01,0.467171,0.296277,7.133009e-01,5.925955e-01,5.777629e-02
75%,44100.0,128111.000000,0.0,60.029366,0.0,-8.969606,44100.000000,8.633399e-01,8.614904e-01,6.528186e-01,7.164915e-01,0.789431,7.083921e-01,6.676705e-01,0.864322,0.663747,9.527178e-01,9.250793e-01,4.718408e-01
max,44100.0,320000.000000,0.0,348.003235,0.0,38.401222,44100.000000,9.999999e-01,9.999861e-01,9.999999e-01,9.999999e-01,0.999930,9.999999e-01,9.999994e-01,1.000000,0.996221,1.000000e+00,9.999841e-01,1.000000e+00


In [ ]:
[audio.columns[i] for i in range(0,len(audio.columns))]

['analysis_sample_rate',
 'bit_rate',
 'equal_loudness',
 'length',
 'lossless',
 'replay_gain',
 'sample_rate',
 'codec',
 'downmix',
 'md5_encoded',
 'track_id',
 'danceability',
 'femaleness',
 'acousticness',
 'aggressiveness',
 'electronicity',
 'happiness',
 'festiveness',
 'relaxation',
 'sadness',
 'brightness',
 'tonality',
 'insrumentalness']

In [ ]:
audio.drop(['analysis_sample_rate','bit_rate','equal_loudness','lossless','replay_gain','sample_rate','codec','downmix','md5_encoded'],inplace=True,axis=1)

# Song

In [ ]:
song.head(3)

,track_id,main_genre
0,TRSZBVQ128F425D5B6,Rock
1,TRRJNCE128F9319E52,RnB
2,TRIZJQX128F1472163,Folk


In [ ]:
song['track_id']=song['track_id'].map(lambda x:x.strip())

In [ ]:
song.isna().mean()

track_id      0.0
main_genre    0.0
dtype: float64

In [ ]:
song.describe()

,track_id,main_genre
count,8416,8416
unique,8416,15
top,TRKLXOY128F931F98A,Rock
freq,1,3413


In [ ]:
from sklearn.neighbors import DistanceMetric
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
song["main_genre"] = song["main_genre"].str.replace("|", " ").str.replace("-", "").str.lower()

In [ ]:
song["main_genre"] 

0             rock
1              rnb
2             folk
3              rap
4       electronic
           ...    
8411         blues
8412          rock
8413          punk
8414          rock
8415         blues
Name: main_genre, Length: 8416, dtype: object

In [ ]:
cv = CountVectorizer(ngram_range=(1,1), min_df=1, max_df=1.0)

In [ ]:
cv.fit(song["main_genre"] )

CountVectorizer()

In [ ]:
genres = pd.DataFrame(data=cv.transform(song["main_genre"]).todense(), columns=[f"genre_{x}" for x in cv.get_feature_names()])

In [ ]:
genres

,genre_age,genre_blues,genre_country,genre_electronic,genre_folk,genre_jazz,genre_latin,genre_metal,genre_new,genre_pop,genre_punk,genre_rap,genre_reggae,genre_rnb,genre_rock,genre_world
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8411,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8412,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8413,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
genres.shape

(8416, 16)

In [ ]:
 song= song.drop(columns = ["main_genre"]).join(genres)

In [ ]:
song.head(3)

,track_id,genre_age,genre_blues,genre_country,genre_electronic,genre_folk,genre_jazz,genre_latin,genre_metal,genre_new,genre_pop,genre_punk,genre_rap,genre_reggae,genre_rnb,genre_rock,genre_world
0,TRSZBVQ128F425D5B6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,TRRJNCE128F9319E52,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,TRIZJQX128F1472163,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


# Summary

In [ ]:
summary.head(3)

,track_id,song_id,mode_confidence,end_of_fade_in,key_confidence,energy,year,duration,time_signature_confidence,title,start_of_fade_out,analysis_sample_rate,time_signature,tempo,artist_id,artist_name,artist_hotttnesss,key,release,loudness
0,TRJPTGQ128F42A2187,SOIYHHS12A8C13E7B5,0.452,0.0,0.556,0.0,2007,141.66159,1.000,Don't Cry,141.662,22050,4,103.085,ARJ5FD21187FB43F12,Mavado,0.520784,0,Gangsta For Life: The Symphony Of David Brooks,-5.816
1,TRZWFET128F425BF9E,SONEJET12A8C137D3E,0.669,0.0,0.679,0.0,2005,354.69016,0.529,Gypsy Funeral: Hark! From The Tombs Of Doleful...,348.253,22050,1,123.153,ARH3CTU1187FB365C7,Miasma & The Carousel Of Headless Horses,0.346908,9,Perils,-11.137
2,TRNTAMF128F14618CA,SOPAGVA12AF729F018,0.551,0.0,0.672,0.0,2001,338.44200,1.000,Trade Money,327.529,22050,4,162.540,AR0CANF1187B9AF35F,Dilated Peoples,0.457841,7,Expansion Team,-5.199


In [ ]:
summary.shape

(20000, 20)

In [ ]:
summary['track_id']=summary['track_id'].map(lambda x:x.strip())
summary['artist_name']=summary['artist_name'].map(lambda x:x.strip())
summary['title']=summary['title'].map(lambda x:x.strip())
summary['title']=summary['title'].map(lambda x:text_clean(x))
summary['artist_name']=summary['artist_name'].map(lambda x:text_clean(x))

In [ ]:
summary.isna().mean()

track_id                     0.0
song_id                      0.0
mode_confidence              0.0
end_of_fade_in               0.0
key_confidence               0.0
energy                       0.0
year                         0.0
duration                     0.0
time_signature_confidence    0.0
title                        0.0
start_of_fade_out            0.0
analysis_sample_rate         0.0
time_signature               0.0
tempo                        0.0
artist_id                    0.0
artist_name                  0.0
artist_hotttnesss            0.0
key                          0.0
release                      0.0
loudness                     0.0
dtype: float64

In [ ]:
summary.columns

Index(['track_id', 'song_id', 'mode_confidence', 'end_of_fade_in',
       'key_confidence', 'energy', 'year', 'duration',
       'time_signature_confidence', 'title', 'start_of_fade_out',
       'analysis_sample_rate', 'time_signature', 'tempo', 'artist_id',
       'artist_name', 'artist_hotttnesss', 'key', 'release', 'loudness'],
      dtype='object')

In [ ]:
summary.describe()

,mode_confidence,end_of_fade_in,key_confidence,energy,year,duration,time_signature_confidence,start_of_fade_out,analysis_sample_rate,time_signature,tempo,artist_hotttnesss,key,loudness
count,20000.000000,20000.000000,20000.000000,20000.0,20000.000000,20000.000000,20000.000000,20000.000000,20000.0,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.473594,0.881743,0.441724,0.0,1998.411700,245.849599,0.514426,236.746542,22050.0,3.610250,125.139885,0.415799,5.315800,-9.669730
std,0.193258,2.871986,0.276916,0.0,11.012046,110.548106,0.370453,108.129202,0.0,1.200382,34.942407,0.110425,3.591395,4.890132
min,0.000000,0.000000,0.000000,0.0,1927.000000,0.783220,0.000000,0.783000,22050.0,0.000000,0.000000,0.000000,0.000000,-52.251000
25%,0.358000,0.000000,0.211000,0.0,1994.000000,183.196280,0.131000,175.216500,22050.0,3.000000,99.916250,0.364310,2.000000,-12.178000
50%,0.485000,0.194500,0.462000,0.0,2002.000000,229.589750,0.552000,220.140000,22050.0,4.000000,122.983000,0.409428,5.000000,-8.624000
75%,0.604000,0.438000,0.652250,0.0,2006.000000,287.555460,0.864000,277.503250,22050.0,4.000000,145.968250,0.469025,9.000000,-6.091750
max,1.000000,235.944000,1.000000,0.0,2010.000000,2342.765260,1.000000,2190.425000,22050.0,7.000000,268.092000,1.082503,11.000000,2.453000


In [ ]:
summary.drop(['song_id','mode_confidence','key_confidence','time_signature_confidence','analysis_sample_rate','artist_id','key','release'],inplace=True,axis=1)

## Cruce

In [ ]:
summary['title-artist']=summary['title']+'-'+summary['artist_name']

In [ ]:
aux=pd.merge(summary,audio,how='inner',on='track_id')

In [ ]:
aux.shape

(17108, 26)

In [ ]:
aux.head(3)

,track_id,end_of_fade_in,energy,year,duration,title,start_of_fade_out,time_signature,tempo,artist_name,...,acousticness,aggressiveness,electronicity,happiness,festiveness,relaxation,sadness,brightness,tonality,insrumentalness
0,TRJPTGQ128F42A2187,0.0,0.0,2007,141.66159,don t cry,141.662,4,103.085,mavado,...,0.022293,0.500000,0.882711,0.456452,0.772325,0.508476,0.067847,0.878608,0.045879,0.007640
1,TRZWFET128F425BF9E,0.0,0.0,2005,354.69016,gypsy funeral hark from the tombs of doleful s...,348.253,1,123.153,miasma the carousel of headless horses,...,0.003386,0.918240,0.395611,0.110496,0.518365,0.262771,0.067540,0.278791,0.150650,0.798211
2,TRNTAMF128F14618CA,0.0,0.0,2001,338.44200,trade money,327.529,4,162.540,dilated peoples,...,0.028752,0.471321,0.934747,0.484657,0.726565,0.012293,0.242436,0.945652,0.123880,0.315464


In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
lst=[aux.columns[i] for i in range(0,len(aux.columns))]

In [ ]:
lst.remove('track_id')
lst.remove('title')
lst.remove('artist_name')
lst.remove('title-artist')

In [ ]:
X = min_max_scaler.fit_transform(aux[lst])

In [ ]:
X

array([[0.        , 0.        , 0.96385542, ..., 0.87860792, 0.04588022,
        0.00763973],
       [0.        , 0.        , 0.93975904, ..., 0.27879085, 0.15065245,
        0.79821066],
       [0.        , 0.        , 0.89156627, ..., 0.9456521 , 0.12388213,
        0.31546353],
       ...,
       [0.00819686, 0.        , 0.85542169, ..., 0.86036847, 0.97505297,
        0.0029758 ],
       [0.        , 0.        , 0.93975904, ..., 0.03060165, 0.09813674,
        0.05533157],
       [0.        , 0.        , 0.84337349, ..., 0.31429897, 0.96994328,
        0.11797363]])

# Distancias sklearn.neighbors.DistanceMetric

## $\bullet$ Distance Metric

In [ ]:
def dist_pairwise(a, b):
    return (a-b)**2

def dist(a,b):
    return np.sqrt(sum([dist_pairwise(x, y) for x,y in zip(a,b)]))

In [ ]:
distance = DistanceMetric.get_metric("euclidean")
X_dist = pd.DataFrame(distance.pairwise(X))

In [ ]:
X_dist

,0,1,2,3,4,5,6,7,8,9,...,17098,17099,17100,17101,17102,17103,17104,17105,17106,17107
0,0.000000,1.612453,0.693058,1.341001,1.766796,1.531823,2.161499,1.822284,1.687858,2.278925,...,1.987314,1.943101,2.441297,1.277846,1.797584,1.692162,2.284040,1.206786,1.342348,1.781915
1,1.612453,0.000000,1.497114,1.883554,1.597811,1.153054,1.587287,1.223950,1.817401,2.277334,...,1.753150,1.696936,2.208336,1.573504,2.093946,0.976043,2.045695,1.840272,1.188021,1.402997
2,0.693058,1.497114,0.000000,1.271086,1.810295,1.688495,1.997252,1.778489,1.735334,2.340078,...,2.084431,1.916210,2.477942,1.471120,1.668503,1.526401,2.275932,1.201657,1.344677,1.659133
3,1.341001,1.883554,1.271086,0.000000,1.831689,1.958650,1.914779,1.766062,1.788668,1.927723,...,1.950046,1.487502,2.095625,1.726984,1.108941,1.665219,1.967468,1.196809,1.409551,1.641420
4,1.766796,1.597811,1.810295,1.831689,0.000000,1.800589,1.550935,2.101183,1.577036,1.580276,...,0.997389,1.030259,1.500777,0.976966,1.921582,1.898995,1.315150,1.930671,1.634703,1.919173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17103,1.692162,0.976043,1.526401,1.665219,1.898995,1.364737,1.477792,0.836950,1.780734,2.300249,...,2.001619,1.636398,2.278543,1.817844,1.890056,0.000000,2.199475,1.568237,1.094897,0.879265
17104,2.284040,2.045695,2.275932,1.967468,1.315150,2.492028,1.506548,2.254815,1.441756,0.856970,...,1.273035,1.147137,0.781048,1.667075,1.579623,2.199475,0.000000,1.990827,2.259527,2.074138
17105,1.206786,1.840272,1.201657,1.196809,1.930671,1.988114,1.615302,1.545584,1.278318,1.910758,...,2.116315,1.583849,2.058435,1.716113,1.436421,1.568237,1.990827,0.000000,1.755857,1.284334
17106,1.342348,1.188021,1.344677,1.409551,1.634703,0.976974,1.809097,1.264961,1.954146,2.262130,...,1.706545,1.589563,2.283085,1.392076,1.798158,1.094897,2.259527,1.755857,0.000000,1.442302


In [ ]:
def buscador(cancion,vecinos):
    index = aux[aux['title-artist'] == cancion].index.values[0]
    ls_closest = X_dist.loc[X_dist.index != index, [index]].sort_values(by = index).head(vecinos).index.tolist()
    return aux.loc[ls_closest, "title-artist"].tolist()

In [ ]:
buscador('umbrella-rihanna jay z',5)

['chocolate radio edit-kylie minogue',
 'kopf oder zahl robot koch dirty sneakers remix-jennifer rostock',
 'hypnotic tango-master blaster',
 'what a girl wants-christina aguilera',
 'disappoint-assemblage 23']

## $\bullet$ sklearn

In [ ]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='euclidean', algorithm='brute', n_neighbors=6, n_jobs=-1)

In [ ]:
model_knn.fit(X)

NearestNeighbors(algorithm='brute', metric='euclidean', n_jobs=-1,
                 n_neighbors=6)

In [ ]:
distances, indices = model_knn.kneighbors(X)

In [ ]:
distances

array([[0.        , 0.40199802, 0.44097436, 0.47232311, 0.48280916,
        0.48456294],
       [0.        , 0.57383497, 0.58510447, 0.59344368, 0.59862051,
        0.60228542],
       [0.        , 0.50510739, 0.5138474 , 0.51518684, 0.52710755,
        0.53537708],
       ...,
       [0.        , 0.37195483, 0.42347839, 0.42820122, 0.4303565 ,
        0.43252473],
       [0.        , 0.38346786, 0.41008006, 0.41196194, 0.42193785,
        0.43385961],
       [0.        , 0.28237379, 0.30201235, 0.38776897, 0.43866883,
        0.45736409]])

In [ ]:
indices

array([[    0, 12353, 12058,  4581,  2171,  1168],
       [    1, 16443, 16612,  8151, 17094, 14505],
       [    2,  9589,  4406,  5495, 10480, 13538],
       ...,
       [17105,  4560, 12801,  3529,  5022, 11801],
       [17106,  2880,  4423,  6945,  6611,  9625],
       [17107, 14340,  2805,  1364,  1837,  4295]], dtype=int64)

### Alternativamente, considere construir un recomendador utilizando sklearn.neighbors.NearestNeighbors utilizando diferentes m ́etricas de distancia (Manhattan, Euclidiana, Similitud Coseno, etc) tomando como referencia 5 vecinos.

In [ ]:
ls_distance_metrics = ["euclidean", "manhattan", "chebyshev", "minkowski"]

In [ ]:
lista=[]
for metric in ls_distance_metrics:
    print(metric)
    model_knn = NearestNeighbors(metric=metric, algorithm='brute', n_neighbors=6, n_jobs=-1)
    model_knn.fit(X)
    lista.append(model_knn.kneighbors(X))

euclidean
manhattan
chebyshev
minkowski


## Construya una función de recomendación que dada la canción y el número de recomendaciones retorne las mejores recomendaciones.

In [ ]:
def buscador(cancion,distancia):
    index = aux[aux['title-artist'] == cancion].index.values[0]
    if distancia == "euclidean":
        ls_closest = lista[0][1][index][1:]
    elif distancia == "manhattan":
        ls_closest = lista[1][1][index][1:]
    elif distancia == "chebyshev":
        ls_closest = lista[2][1][index][1:]
    elif distancia == "minkowski":
        ls_closest = lista[3][1][index][1:]
        
    return aux.loc[ls_closest, "title-artist"].tolist()

In [ ]:
buscador('umbrella-rihanna jay z',distancia ='euclidean')

['chocolate radio edit-kylie minogue',
 'kopf oder zahl robot koch dirty sneakers remix-jennifer rostock',
 'hypnotic tango-master blaster',
 'what a girl wants-christina aguilera',
 'disappoint-assemblage 23']

In [ ]:
buscador('umbrella-rihanna jay z',distancia ='chebyshev')

['chocolate radio edit-kylie minogue',
 'going wrong-armin van buuren',
 'we are the people we are the cagedbaby mix uk edit-empire of the sun',
 'invisible-jennifer hudson',
 'get me bodied-beyonc xc3 xa9']

In [ ]:
buscador('umbrella-rihanna jay z',distancia ='manhattan')

['kopf oder zahl robot koch dirty sneakers remix-jennifer rostock',
 'disappoint-assemblage 23',
 'you ll never know what it means-bananarama',
 'chocolate radio edit-kylie minogue',
 'mary-scissor sisters']

In [ ]:
buscador('umbrella-rihanna jay z',distancia ='minkowski')

['chocolate radio edit-kylie minogue',
 'kopf oder zahl robot koch dirty sneakers remix-jennifer rostock',
 'hypnotic tango-master blaster',
 'what a girl wants-christina aguilera',
 'disappoint-assemblage 23']

In [ ]:
buscador('umbrella-rihanna jay z',distancia ='euclidean')

# ¿Las recomendaciones son distintas?

 ##  R = Para las métricas euclidean y minkowski las recomendaciones son iguales pero para las distancias manhattan y chebyshev los conjuntos son diferentes

   # ¿A qué se debe dicho comportamiento?

# R = A que la construcción del algoritmo dada la distancia hace que varien las recomendaciones

# ¿Cuál es el mejor recomendador? y  ¿Bajo que criterio se eligió un recomendador sobre otro?

## Definición de criterio de selección

## El racional del score de canción se define como la frecuecia de aparición de la canción en los distintos conjuntos generados a partir de las métricas antes definidas.

In [ ]:
puntaje=pd.DataFrame(buscador('umbrella-rihanna jay z',distancia ='euclidean')+buscador('umbrella-rihanna jay z',distancia ='manhattan')+buscador('umbrella-rihanna jay z',distancia ='minkowski')+buscador('umbrella-rihanna jay z',distancia ='chebyshev'),columns=['Canciones']).value_counts()

In [ ]:
puntaje.name='puntaje'

In [ ]:
euclidean=pd.DataFrame(buscador('umbrella-rihanna jay z',distancia ='euclidean'),columns=['Canciones'])
pd.merge(euclidean,puntaje,how='inner',on='Canciones')['puntaje'].sum()

14

In [ ]:
minkowski=pd.DataFrame(buscador('umbrella-rihanna jay z',distancia ='minkowski'),columns=['Canciones'])
pd.merge(minkowski,puntaje,how='inner',on='Canciones')['puntaje'].sum()

14

In [ ]:
chebyshev=pd.DataFrame(buscador('umbrella-rihanna jay z',distancia ='chebyshev'),columns=['Canciones'])
pd.merge(chebyshev,puntaje,how='inner',on='Canciones')['puntaje'].sum()

8

In [ ]:
manhattan=pd.DataFrame(buscador('umbrella-rihanna jay z',distancia ='manhattan'),columns=['Canciones'])
pd.merge(manhattan,puntaje,how='inner',on='Canciones')['puntaje'].sum()

12

## Conclusión: Notamos que los recomendadores ganadores son los creados a partir de métricas euclideana y minkowski. 